Speeches folder path: /Users/mason/Desktop/Technical_Projects/PYTHON_Projects/PSAI/raw/Speeches

Speeches:

Title: America Truly is the Greatest Country in the World. Don't Let Freedom Slip Away
- Speaker: Kitty Werthman
- /Users/mason/Desktop/Technical_Projects/PYTHON_Projects/PSAI/raw/Speeches/America Truly is the Greatest Country in the World. Don't Let Freedom Slip Away  by Kitty Werthmann.pdf

Title: Don't Be Depressed About America
- Speaker: Phyllis Schlafly
- Date: 1973-05-24
- /Users/mason/Desktop/Technical_Projects/PYTHON_Projects/PSAI/raw/Speeches/DON'T BE DEPRESSED ABOUT AMERICA PS WBBM Radio commentary 5-24-1973.pdf

Title: Fred Schlafly on Roe V. Wade The Supreme Court decision in favor of abortion
- Speaker: Fred Schlafly
- Date: 1973-01-23
- /Users/mason/Desktop/Technical_Projects/PYTHON_Projects/PSAI/raw/Speeches/Fred Schlafly on Roe V. Wade The Supreme Court decision in favor of abortion January 23, 1973.pdf

Title: The Ten Skills of Leadership
- Speaker: Phyllis Schlafly
- /Users/mason/Desktop/Technical_Projects/PYTHON_Projects/PSAI/raw/Speeches/Leadership by PS .pdf

Title: PS Introduction of Sen. Robert Taft in Alton, IL
- Speaker: Phyllis Schlafly
- Date: 1952-04-04
- /Users/mason/Desktop/Technical_Projects/PYTHON_Projects/PSAI/raw/Speeches/PS Introduction of Sen. Robert Taft in Alton, IL 4-4-1952.pdf

Title: The Big Things Are Done By Little People by Phyllis Schlafly
- Speaker: Phyllis Schlafly
- /Users/mason/Desktop/Technical_Projects/PYTHON_Projects/PSAI/raw/Speeches/THE BlG THINGS ARE DONE BY LITTLE PEOPLE by Phyllis Schlafly PS.pdf

Title: The Surest Way To Peace by Phyllis Schlafly
- Speaker: Phyllis Schlafly
- Date: 1966-10-14
- /Users/mason/Desktop/Technical_Projects/PYTHON_Projects/PSAI/raw/Speeches/THE SUREST WAY TO PEACE speech by PS to AAPS 10-14-1966.pdf

Title: Winning Word Warfare In Politics by Phyllis Schlafly
- Speaker: Phyllis Schlafly
- Date: 1968-02-15
- /Users/mason/Desktop/Technical_Projects/PYTHON_Projects/PSAI/raw/Speeches/WINNING WORD WARFARE IN POLITICS PS Speech to The Eagles Are Flying 2-15-1968.pdf

In [1]:
# Imports

import os
import json
import uuid
import fitz
from pathlib import Path
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct, VectorParams, Distance

/Users/mason/opt/anaconda3/envs/psai/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
OUTPUT_DIR = Path("/Users/mason/Desktop/Technical_Projects/PYTHON_Projects/PSAI/chunks/batch5/speeches")
CHUNK_SIZE = 1000
COLLECTION_NAME = "speeches_chunks"
load_dotenv("/Users/mason/Desktop/Technical_Projects/PYTHON_Projects/PSAI/code/.env")

# Create output directory if it doesn't exist
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Define the speeches with their metadata
SPEECHES = [
    (
        "America Truly is the Greatest Country in the World. Don't Let Freedom Slip Away",
        "Kitty Werthmann", 
        None,
        "/Users/mason/Desktop/Technical_Projects/PYTHON_Projects/PSAI/raw/Speeches/America Truly is the Greatest Country in the World. Don't Let Freedom Slip Away  by Kitty Werthmann.pdf"
    ),
    (
        "Don't Be Depressed About America",
        "Phyllis Schlafly",
        "1973-05-24",
        "/Users/mason/Desktop/Technical_Projects/PYTHON_Projects/PSAI/raw/Speeches/DON'T BE DEPRESSED ABOUT AMERICA PS WBBM Radio commentary 5-24-1973.pdf"
    ),
    (
        "Fred Schlafly on Roe V. Wade The Supreme Court decision in favor of abortion",
        "Fred Schlafly",
        "1973-01-23", 
        "/Users/mason/Desktop/Technical_Projects/PYTHON_Projects/PSAI/raw/Speeches/Fred Schlafly on Roe V. Wade The Supreme Court decision in favor of abortion January 23, 1973.pdf"
    ),
    (
        "The Ten Skills of Leadership",
        "Phyllis Schlafly",
        None,
        "/Users/mason/Desktop/Technical_Projects/PYTHON_Projects/PSAI/raw/Speeches/Leadership by PS .pdf"
    ),
    (
        "PS Introduction of Sen. Robert Taft in Alton, IL",
        "Phyllis Schlafly",
        "1952-04-04",
        "/Users/mason/Desktop/Technical_Projects/PYTHON_Projects/PSAI/raw/Speeches/PS Introduction of Sen. Robert Taft in Alton, IL 4-4-1952.pdf"
    ),
    (
        "The Big Things Are Done By Little People by Phyllis Schlafly",
        "Phyllis Schlafly",
        None,
        "/Users/mason/Desktop/Technical_Projects/PYTHON_Projects/PSAI/raw/Speeches/THE BlG THINGS ARE DONE BY LITTLE PEOPLE by Phyllis Schlafly PS.pdf"
    ),
    (
        "The Surest Way To Peace by Phyllis Schlafly", 
        "Phyllis Schlafly",
        "1966-10-14",
        "/Users/mason/Desktop/Technical_Projects/PYTHON_Projects/PSAI/raw/Speeches/THE SUREST WAY TO PEACE speech by PS to AAPS 10-14-1966.pdf"
    ),
    (
        "Winning Word Warfare In Politics by Phyllis Schlafly",
        "Phyllis Schlafly", 
        "1968-02-15",
        "/Users/mason/Desktop/Technical_Projects/PYTHON_Projects/PSAI/raw/Speeches/WINNING WORD WARFARE IN POLITICS PS Speech to The Eagles Are Flying 2-15-1968.pdf"
    )
]

In [3]:
# Helper Functions

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file."""
    doc = fitz.open(pdf_path)
    text = "\n".join(page.get_text() for page in doc)
    doc.close()
    return text

def chunk_text(text, chunk_size=1000):
    """Split text into chunks of approximately chunk_size characters."""
    paragraphs = text.split("\n")
    chunks, current_chunk = [], ""
    for para in paragraphs:
        if not para.strip():
            continue
        if len(current_chunk) + len(para) < chunk_size:
            current_chunk += para.strip() + " "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = para.strip() + " "
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

def build_json_chunks(chunks, title, speaker, date):
    """Build JSON chunks with metadata for each text chunk."""
    return [
        {
            "speaker": speaker,
            "title": title,
            "date": date,
            "text": chunk
        }
        for chunk in chunks
    ]

def save_chunks_to_json(data, output_path):
    """Save chunks to a JSON file."""
    with open(output_path, "w") as f:
        json.dump(data, f, indent=2)

# Process all speeches
all_chunks = []
for title, speaker, date, path in SPEECHES:
    print(f"Processing: {title} by {speaker}")
    
    # Check if file exists
    if not Path(path).exists():
        print(f"Warning: File not found - {path}")
        continue
    
    try:
        text = extract_text_from_pdf(path)
        chunks = chunk_text(text, CHUNK_SIZE)
        json_chunks = build_json_chunks(chunks, title, speaker, date)
        
        # Create a safe filename
        safe_filename = Path(path).stem.replace(" ", "_").replace("'", "")
        outpath = OUTPUT_DIR / (safe_filename + ".json")
        save_chunks_to_json(json_chunks, outpath)
        
        all_chunks.extend(json_chunks)
        print(f"  - Created {len(chunks)} chunks")
        
    except Exception as e:
        print(f"Error processing {path}: {str(e)}")
        continue

print(f"\nTotal chunks prepared: {len(all_chunks)}")



Processing: America Truly is the Greatest Country in the World. Don't Let Freedom Slip Away by Kitty Werthmann
  - Created 13 chunks
Processing: Don't Be Depressed About America by Phyllis Schlafly
  - Created 2 chunks
Processing: Fred Schlafly on Roe V. Wade The Supreme Court decision in favor of abortion by Fred Schlafly
  - Created 2 chunks
Processing: The Ten Skills of Leadership by Phyllis Schlafly
  - Created 2 chunks
Processing: PS Introduction of Sen. Robert Taft in Alton, IL by Phyllis Schlafly
  - Created 4 chunks
Processing: The Big Things Are Done By Little People by Phyllis Schlafly by Phyllis Schlafly
  - Created 10 chunks
Processing: The Surest Way To Peace by Phyllis Schlafly by Phyllis Schlafly
  - Created 35 chunks
Processing: Winning Word Warfare In Politics by Phyllis Schlafly by Phyllis Schlafly
  - Created 44 chunks

Total chunks prepared: 112


In [4]:
# Embedding and upload to Qdrant

model = SentenceTransformer("all-MiniLM-L6-v2")
qdrant = QdrantClient(
    url=os.getenv("QDRANT_URL"),
    api_key=os.getenv("QDRANT_API_KEY")
)

def ensure_collection(name):
    """Ensure the collection exists, create if it doesn't."""
    collections = qdrant.get_collections().collections
    if not any(c.name == name for c in collections):
        qdrant.recreate_collection(
            collection_name=name,
            vectors_config=VectorParams(size=384, distance=Distance.COSINE)
        )
        print(f"Created new collection: {name}")

def embed_and_upload(chunks, collection_name):
    """Generate embeddings and upload to Qdrant."""
    if not chunks:
        print("No chunks to upload!")
        return
        
    ensure_collection(collection_name)
    texts = [c["text"] for c in chunks]
    
    print("Generating embeddings...")
    vectors = model.encode(texts).tolist()
    payloads = chunks
    
    points = [
        PointStruct(id=str(uuid.uuid4()), vector=vec, payload=payload)
        for vec, payload in zip(vectors, payloads)
    ]
    
    print("Uploading to Qdrant...")
    qdrant.upload_points(collection_name=collection_name, points=points)
    print(f"Uploaded {len(points)} points to '{collection_name}'")

embed_and_upload(all_chunks, COLLECTION_NAME)
print("Done!")


/var/folders/fc/s2pyk63x5958wcxdtr6xlpwc0000gn/T/ipykernel_44256/4243596192.py:13: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


Created new collection: speeches_chunks
Generating embeddings...
Uploading to Qdrant...
Uploaded 112 points to 'speeches_chunks'
Done!
